In [1]:
import os
import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 [==============================] - 1s 0us/step


In [3]:
#definicja modelu
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Trening modelu z zapisem punktów kontrolnych (checkpoints)

In [4]:
checkpoints_path = "training_1/cp.ckpt"
checkpoints_dir = os.path.dirname(checkpoints_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoints_path,
                                                 save_weights_only = True,
                                                 verbose = 1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])

Epoch 1/10
30/32 [===========================>..] - ETA: 0s - loss: 1.1772 - sparse_categorical_accuracy: 0.6833
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 19ms/step - loss: 1.1555 - sparse_categorical_accuracy: 0.6890 - val_loss: 0.7161 - val_sparse_categorical_accuracy: 0.7730
Epoch 2/10
24/32 [=====================>........] - ETA: 0s - loss: 0.4097 - sparse_categorical_accuracy: 0.8815
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.4088 - sparse_categorical_accuracy: 0.8830 - val_loss: 0.5847 - val_sparse_categorical_accuracy: 0.8150
Epoch 3/10
25/32 [======================>.......] - ETA: 0s - loss: 0.3112 - sparse_categorical_accuracy: 0.9212
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.2996 - sparse_categorical_accuracy: 0.9230 - val_loss: 0.5568 - val_sparse_categorical_accuracy: 0.8180
Epoch 4/10
25/32 [=======

In [5]:
os.listdir(checkpoints_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [6]:
model = create_model()
loss, acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"niewytrenowany model -> dopasowanie: {100*acc:.2f}%")

32/32 - 0s - loss: 2.4086 - sparse_categorical_accuracy: 0.0290 - 180ms/epoch - 6ms/step
niewytrenowany model -> dopasowanie: 2.90%


In [7]:
model.load_weights(checkpoints_path)
loss, acc = model.evaluate(test_images,test_labels,verbose=2)
print(f"odtworzony model -> dopasowanie: {100*acc:.2f}%")

32/32 - 0s - loss: 0.4108 - sparse_categorical_accuracy: 0.8670 - 75ms/epoch - 2ms/step
odtworzony model -> dopasowanie: 86.70%
